# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port alfred,-33.5906,26.8910,16.56,80,0,3.73,ZA,1697593373
1,1,waitangi,-43.9535,-176.5597,12.68,72,27,2.24,NZ,1697593374
2,2,avarua,-21.2078,-159.7750,25.03,61,40,7.20,CK,1697593374
3,3,saint-philippe,-21.3585,55.7679,22.77,77,100,2.32,RE,1697593375
4,4,punta arenas,-53.1500,-70.9167,9.06,71,75,6.17,CL,1697593376


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )
# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 26.5)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] < 3)
                                       & (city_data_df["Wind Speed"] < 4.15)
                                      ]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
91,91,al hufuf,25.3646,49.5653,25.19,69,0,1.54,SA,1697593456
95,95,tura,25.5198,90.2201,23.40,77,0,0.38,IN,1697593459
148,148,remire-montjoly,4.9167,-52.2667,26.02,89,0,1.54,GF,1697593504
183,183,muroto-misakicho,33.2833,134.1500,21.74,60,0,1.56,JP,1697593535
219,219,port lincoln,-34.7333,135.8667,21.44,44,0,3.49,AU,1697593572


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
91,al hufuf,SA,25.3646,49.5653,69,
95,tura,IN,25.5198,90.2201,77,
148,remire-montjoly,GF,4.9167,-52.2667,89,
183,muroto-misakicho,JP,33.2833,134.1500,60,
219,port lincoln,AU,-34.7333,135.8667,44,
237,tazacorte,ES,28.6290,-17.9293,79,
243,aswan,EG,24.0934,32.9070,40,
255,naze,JP,28.3667,129.4833,54,
266,al 'amarah,IQ,31.8406,47.1396,36,
273,badr hunayn,SA,23.7829,38.7905,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 5000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
al hufuf - nearest hotel: Al Muhaidab Residence Al Ahsa
tura - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
muroto-misakicho - nearest hotel: 民宿うらしま
port lincoln - nearest hotel: Boston Hotel
tazacorte - nearest hotel: App Leyma
aswan - nearest hotel: Yaseen hotel
naze - nearest hotel: ビジネス旅館畠山
al 'amarah - nearest hotel: فندق الكورميك التركي
badr hunayn - nearest hotel: No hotel found
timimoun - nearest hotel: أناف
rouissat - nearest hotel: فندق الأروي الذهبي
nova sintra - nearest hotel: Residência Ka Dencho
matoury - nearest hotel: La Marmotte
al musayyib - nearest hotel: عمارة الصفار
strathalbyn - nearest hotel: No hotel found
aldinga beach - nearest hotel: No hotel found
ibara - nearest hotel: ビジネスホテル 歴城荘
dwarka - nearest hotel: The Dwarika Hotel
shimoda - nearest hotel: ホテルウラガ
as sulayyil - nearest hotel: No hotel found
kunri - nearest hotel: No hotel found
melilla - nearest hotel: Rusadir


,City,Country,Lat,Lng,Humidity,Hotel Name
91,al hufuf,SA,25.3646,49.5653,69,Al Muhaidab Residence Al Ahsa
95,tura,IN,25.5198,90.2201,77,No hotel found
148,remire-montjoly,GF,4.9167,-52.2667,89,Complexe Belova
183,muroto-misakicho,JP,33.2833,134.1500,60,民宿うらしま
219,port lincoln,AU,-34.7333,135.8667,44,Boston Hotel
237,tazacorte,ES,28.6290,-17.9293,79,App Leyma
243,aswan,EG,24.0934,32.9070,40,Yaseen hotel
255,naze,JP,28.3667,129.4833,54,ビジネス旅館畠山
266,al 'amarah,IQ,31.8406,47.1396,36,فندق الكورميك التركي
273,badr hunayn,SA,23.7829,38.7905,54,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)